In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nannyml as nml
from scipy.spatial.transform import Rotation

# 10 reference periods
# 10 analysis periods
# Days/week * Hours/day * events/hour
DPP = 7*24*12

np.random.seed(23)
s1 = np.random.randn(DPP*20)
x1 = s1 + np.random.randn(DPP*20)/8
x2 = s1 + np.random.randn(DPP*20)/8
x3 = np.random.randn(DPP*20)/8
xdat = np.array([x1, x2, x3]).T

rot = Rotation.from_euler('z', 90, degrees=True)

# following matrix multiplication implementation, we need a 3xN data matrix hence we transpose
ydat = np.matmul(rot.as_matrix(), xdat.T).T

# create overall array that has drifted and not drifted subsets.
# drift is sudden and affects last 5 weeks
dataar = np.concatenate(
    (xdat[:-5*DPP], ydat[-5*DPP:]),
    axis=0
)

# convert data to dataframe
datadf = pd.DataFrame(dataar, columns=['feature1', 'feature2', 'feature3'])

# add "timestamp" column
datadf = datadf.assign(ordered = pd.date_range(start='1/6/2020', freq='5min', periods=20*DPP))

# Adding helper column - duplicates date range functionality
datadf['week'] = datadf.ordered.dt.isocalendar().week - 1
# Adding partition column
datadf['partition'] = 'reference'
datadf.loc[datadf.week >= 11, ['partition']] = 'analysis'

# Assign random predictions and targets (we won't be using them but they are needed for NannyML)
datadf = datadf.assign(y_pred_proba = np.random.rand(DPP*20))
datadf = datadf.assign(y_true = np.random.randint(2, size=DPP*20))

In [ ]:
# let's construct a dataframe for visuzlization purposes
dat1 = datadf.loc[datadf.week == 10, ['feature1', 'feature2']][:1500]
dat1['week'] = 10
dat2 = datadf.loc[datadf.week == 16, ['feature1', 'feature2']][:1500]
dat2['week'] = 16
data_sample = pd.concat([dat1, dat2], ignore_index=True)

# let's plot
colors = nml.plots.colors.Colors
figure = sns.jointplot(
    data=data_sample,
    x="feature1",
    y="feature2",
    hue="week",
    palette=[colors.BLUE_SKY_CRAYOLA.value, colors.RED_IMPERIAL.value]
)
figure.fig.suptitle('Data Distributions before and after rotation drift')

Text(0.5, 0.98, 'Data Distributions before and after rotation drift')

In [ ]:
datadf

,feature1,feature2,feature3,ordered,week,partition,y_pred_proba,y_true
0,0.776707,0.886555,-0.106081,2020-01-06 00:00:00,1,reference,0.366613,1
1,0.049000,-0.108099,0.121577,2020-01-06 00:05:00,1,reference,0.699295,0
2,-0.982245,-0.653828,-0.073487,2020-01-06 00:10:00,1,reference,0.384817,0
3,1.033905,0.975908,-0.130573,2020-01-06 00:15:00,1,reference,0.813934,1
4,0.550946,0.746063,-0.017051,2020-01-06 00:20:00,1,reference,0.858081,0
...,...,...,...,...,...,...,...,...
40315,1.156739,-1.399472,0.120137,2020-05-24 23:35:00,20,analysis,0.825109,1
40316,-0.483793,0.467371,-0.014811,2020-05-24 23:40:00,20,analysis,0.718616,0
40317,0.500973,-0.612287,-0.104527,2020-05-24 23:45:00,20,analysis,0.537909,1
40318,0.911855,-0.731479,-0.143904,2020-05-24 23:50:00,20,analysis,0.571672,0


In [ ]:
# Let's first create the analysis and reference datasets NannyML needs.
reference = datadf.loc[datadf['partition'] == 'reference'].reset_index(drop=True)
reference.drop(['week', 'partition'], axis=1, inplace=True)
analysis = datadf.loc[datadf['partition'] == 'analysis'].reset_index(drop=True)
analysis.drop(['y_true', 'week', 'partition'], axis=1, inplace=True)

feature_column_names = ['feature1', 'feature2', 'feature3']

# Let's instantiate and calibrate univariate drift
univariate_calculator = nml.UnivariateStatisticalDriftCalculator(
    feature_column_names=feature_column_names,
    timestamp_column_name='ordered',
    chunk_size=DPP
)
univariate_calculator.fit(reference_data=reference)

# let's compute (and visualize) results across all the dataset.
univariate_results = univariate_calculator.calculate(data=analysis)
for feature in feature_column_names:
    figure = univariate_results.plot(
        kind='feature_distribution',
        feature_column_name=feature,
        plot_reference=True
    )
    figure.show()

In [ ]:
# Let's compute multivariate drift
rcerror_calculator = nml.DataReconstructionDriftCalculator(
    feature_column_names=feature_column_names,
    timestamp_column_name='ordered',
    chunk_size=DPP
).fit(reference_data=reference)
# let's compute results for analysis period
rcerror_results = rcerror_calculator.calculate(data=analysis)

# let's visualize results across all the dataset
figure = rcerror_results.plot(plot_reference=True)
figure.show()